In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
neighborhoods['Borough'].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [8]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [9]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Borough'], Brooklyn_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [11]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'H4UYJU3CMIE02IFAHGV3I5DXFUNIQKW3D4QWZLWKGB45EWBT'
CLIENT_SECRET = 'Z4KBLX4LIM3BMHWKU5WBKMGX50TBDIAIXRPMEI20H4FEKYSB'
VERSION = '20181020'

In [12]:
#https://developer.foursquare.com/docs/resources/categories
#Indian = 4bf58dd8d48988d10f941735


neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
newyork_venues_bookstore = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d114951735')
newyork_venues_bookstore.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore
1,Bay Ridge,40.625801,-74.030621,Book Mark Shop,40.623848,-74.031059,Bookstore
2,Bensonhurst,40.611009,-73.995180,Excalibur Comics,40.611408,-73.991451,Bookstore
3,Sunset Park,40.645103,-74.010316,Koch Comics Warehouse,40.653740,-74.011697,Bookstore
4,Sunset Park,40.645103,-74.010316,JC Christian Bookstore,40.640991,-74.014412,Bookstore


In [13]:
newyork_venues_bookstore.shape

(274, 7)

In [14]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [15]:
map_newyork_bookstore = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_bookstore, 'red', map_newyork_bookstore)

map_newyork_bookstore

In [16]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0
Brooklyn_grouped = newyork_venues_bookstore.groupby('Neighborhood').count()
Brooklyn_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Ridge,2,2,2,2,2,2
Bedford Stuyvesant,1,1,1,1,1,1
Bensonhurst,1,1,1,1,1,1
Boerum Hill,22,22,22,22,22,22
Borough Park,8,8,8,8,8,8
Brighton Beach,4,4,4,4,4,4
Brooklyn Heights,13,13,13,13,13,13
Bushwick,7,7,7,7,7,7
Canarsie,1,1,1,1,1,1


In [17]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(newyork_venues_bookstore[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = newyork_venues_bookstore['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Neighborhood,Bike Shop,Bookstore,Coffee Shop,Comic Shop,Cosmetics Shop,Furniture / Home Store,Gift Shop,Library,Miscellaneous Shop,Newsstand,Restaurant,Toy / Game Store,Used Bookstore,Video Store
0,Bay Ridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Bay Ridge,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Bensonhurst,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Sunset Park,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Sunset Park,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Bike Shop,Bookstore,Coffee Shop,Comic Shop,Cosmetics Shop,Furniture / Home Store,Gift Shop,Library,Miscellaneous Shop,Newsstand,Restaurant,Toy / Game Store,Used Bookstore,Video Store
0,Bay Ridge,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
1,Bedford Stuyvesant,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
2,Bensonhurst,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
3,Boerum Hill,0.0,0.818182,0.000000,0.000000,0.045455,0.000000,0.090909,0.000000,0.000,0.045455,0.00,0.000000,0.000000,0.00
4,Borough Park,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
5,Brighton Beach,0.0,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.25
6,Brooklyn Heights,0.0,0.846154,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000,0.076923,0.00,0.000000,0.000000,0.00
7,Bushwick,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
8,Canarsie,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00
9,Carroll Gardens,0.0,0.857143,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
1,Bedford Stuyvesant,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
2,Bensonhurst,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
3,Boerum Hill,Bookstore,Gift Shop,Newsstand,Cosmetics Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Miscellaneous Shop,Library
4,Borough Park,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store


In [22]:
# set number of clusters
kclusters = 5

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([0, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 2, 0, 3, 1, 3, 3, 3, 0, 3, 2, 0, 0, 4, 0, 3, 0, 3, 3, 0,
       0], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



Brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,3.0,Bookstore,Restaurant,Video Store,Used Bookstore,Toy / Game Store,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
4,Brooklyn,Gravesend,40.595260,-73.973471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
Brooklyn_merged=Brooklyn_merged.dropna()
Brooklyn_merged.isna().sum()

Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [25]:
Brooklyn_merged=Brooklyn_merged[Brooklyn_merged['Neighborhood']!='Gravesend']
Brooklyn_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
2,Brooklyn,Sunset Park,40.645103,-74.010316,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,3.0,Bookstore,Restaurant,Video Store,Used Bookstore,Toy / Game Store,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
5,Brooklyn,Brighton Beach,40.576825,-73.965094,3.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,3.0,Bookstore,Miscellaneous Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Library,Gift Shop,Furniture / Home Store
9,Brooklyn,Crown Heights,40.670829,-73.943291,3.0,Bookstore,Coffee Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop
11,Brooklyn,Kensington,40.642382,-73.980421,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
12,Brooklyn,Windsor Terrace,40.656946,-73.980073,0.0,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 0, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
1,Bensonhurst,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
2,Sunset Park,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
6,Sheepshead Bay,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
11,Kensington,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
12,Windsor Terrace,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
16,Bushwick,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
17,Bedford Stuyvesant,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
22,Gowanus,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
28,Canarsie,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store


In [28]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 1, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Marine Park,Used Bookstore,Video Store,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store,Cosmetics Shop


In [29]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 2, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Prospect Heights,Bookstore,Used Bookstore,Coffee Shop,Video Store,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop
68,Madison,Bookstore,Used Bookstore,Video Store,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store


In [30]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Greenpoint,Bookstore,Restaurant,Video Store,Used Bookstore,Toy / Game Store,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
5,Brighton Beach,Bookstore,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop,Furniture / Home Store
7,Manhattan Terrace,Bookstore,Miscellaneous Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Library,Gift Shop,Furniture / Home Store
9,Crown Heights,Bookstore,Coffee Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop
15,Williamsburg,Bookstore,Furniture / Home Store,Comic Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library
18,Brooklyn Heights,Bookstore,Newsstand,Gift Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Miscellaneous Shop,Library,Furniture / Home Store
19,Cobble Hill,Bookstore,Gift Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Furniture / Home Store
20,Carroll Gardens,Bookstore,Gift Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Furniture / Home Store
23,Fort Greene,Bookstore,Cosmetics Shop,Coffee Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library
24,Park Slope,Bookstore,Used Bookstore,Gift Shop,Video Store,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Furniture / Home Store


In [31]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4, Brooklyn_merged.columns[[1] + list(range(5, Brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Red Hook,Bookstore,Bike Shop,Video Store,Used Bookstore,Toy / Game Store,Restaurant,Newsstand,Miscellaneous Shop,Library,Gift Shop
